# LLama3 8B 모델을 Sagemaker를 통해 g5 인스턴스에 배포하기

---

## Ref
- [maum-ai/Llama-3-MAAL-8B-Instruct-v0.1](https://huggingface.co/maum-ai/Llama-3-MAAL-8B-Instruct-v0.1)

---

## 실험 환경
- 이 노트북은 SageMaker Studio Code Editor 및 커널 base (Python 3.10.13) 에서 테스트 되었습니다.

---

# 0. 사전 진행 내용
- Llama3 모델을 사용하기 위해서는 아래의 웹페이지에 가서 본인의 계정으로 로그인 후에 "동의" 를 먼저 해야 합니다.
    - [meta-llama/Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)
- 또한 본의 HF Key 을 얻기 위해서는, [User access tokens](https://huggingface.co/docs/hub/en/security-tokens) 참고 하세요.

# 1. 환경 셋업

상위 폴더의 경로를 추가하여 해당 유틸리티, 이미지 폴더를 참조 합니다.

In [35]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/40_inference is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/40_inference/30-Llama-3-Inference', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/40_inference']


In [2]:
# install_needed = True
install_needed = False

if install_needed:
    ! pip install sagemaker --upgrade  --quiet
    ! pip list | grep -E "sagemaker"

In [3]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## [중요] Hugging Face HF_Key 를 환경변수에 저장 
- 아래에 본인의 Key 를 입력하고, 주석을 제거 후에 사용하세요.

    ```
    key_val = "<Type Your HF Key>"
    # set_hf_key_env_vars(hf_key_name, key_val)
    ```


In [6]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

import os

def set_hf_key_env_vars(hf_key_name, key_val):
    os.environ[hf_key_name] = key_val

def get_hf_key_env_vars(hf_key_name):
    HF_key_value = os.environ.get(hf_key_name)

    return HF_key_value

hf_key_name = "HF_KEY"
key_val = "<Type Your HF Key>"
# set_hf_key_env_vars(hf_key_name, key_val)


HF_key_value = get_hf_key_env_vars(hf_key_name)
# print("HF_key_value: ", HF_key_value)



# 2. HF 파라미터 설정



## 모델 설정

In [7]:
# model id
hf_model_id = 'maum-ai/Llama-3-MAAL-8B-Instruct-v0.1'


## 환경 변수 설정

In [8]:
# instance type
instance_type = "ml.g5.2xlarge"

# Set GPU_NUM
if instance_type == "ml.g5.2xlarge":
    num_gpu = "1"
elif instance_type == "ml.g5.24xlarge":
    num_gpu = "4"
else:
    num_gpu = None

In [9]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID': hf_model_id,
	'SM_NUM_GPUS': num_gpu,
	'HUGGING_FACE_HUB_TOKEN': HF_key_value
}

## 추론 도커 이미지 설정

In [10]:
image_uri = get_huggingface_llm_image_uri("huggingface",version="2.0.2")
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04'

## SageMaker Model 의 하위 클래스인 HuggingFaceModel 생성

In [11]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri= image_uri,
	env=hub,
	role=role, 
)



# 3. SageMaker Endpoint 에 배포 

## endpoint_name 생성

In [14]:
from datetime import datetime

def create_ennpoint_name(model_id, instance_type):

    hf_model_id = model_id.split('/')[1]

    instance_type = instance_type.replace('.','-')
    hf_model_id = hf_model_id.replace('.','-')
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d-%H-%M-%S")
    endpoint_name = f"{hf_model_id}-{instance_type}-{formatted_datetime}"

    return endpoint_name

endpoint_name = create_ennpoint_name(hf_model_id, instance_type)
print("endpoint_name: ", endpoint_name)

    

endpoint_name:  Llama-3-MAAL-8B-Instruct-v0-1-ml-g5-2xlarge-2024-05-19-13-05-07


## SageMaker Endpoint 배포

In [15]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	endpoint_name = endpoint_name,
	initial_instance_count=1,
	instance_type= instance_type,
	container_startup_health_check_timeout=300,
  )

------------!

# 4. 추론

## pay_load 생성

In [56]:
from inference_utils.inference_util import ( print_ww, 
                                            pretty_print_json,
                                            invoke_endpoint_sagemaker
                                       )
                                       
def create_payload_llama_8b(prompt, param):
    # prompt="What is a machine learning?"
    input_data = f"{prompt}"
    pay_load = {"inputs": input_data, "parameters": param}

    # payload_str = json.dumps(pay_load)
    return pay_load
    # return payload_str.encode("utf-8")
    
def llama3_output_parser(response):
    completion = json.loads(response)
    completion = completion[0]
    completion = completion['generated_text']

    return completion


In [59]:
prompt = "Generative AI is"
param = {"do_sample": True, "max_new_tokens": 256}
pay_load = create_payload_llama_8b(prompt, param)


In [63]:
print("## payload: ") 
pretty_print_json(pay_load)


## payload: 
{
    "inputs": "Generative AI is",
    "parameters": {
        "do_sample": true,
        "max_new_tokens": 256
    }
}


## 추론

In [62]:
response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    




response = llama3_output_parser(response)
print_ww(response)



Generative AI is a cutting-edge technique that produces original and tailored images to suit your
request. By comprehending your demands, our infrastructure combines support for multiple formats,
languages, and dialects, enabling your request to be implemented by an AI model with expertise in
the data. With this, we can manufacture dynamic and original images that our first set of models
could not achieve. Our advanced infrastructure guarantees efficient operation in complex inquiries
that necessitate sophisticated image creation. By having this as a foundation, we can furnish you
with images that satisfy both your expectations and needs. In case we fail in your expectations, we
will always be here to resolve and make sure the images provided are the result of your desired
outcome. While considering generative AI as an alternative, it's essential to remember that, due to
its limitations, generative AI may not always reflect your ideas perfectly. However, we can evolve
and add functiona

## 한글 추론

In [64]:
prompt = "생성형 AI 는"
param = {"do_sample": True, "max_new_tokens": 256}
pay_load = create_payload_llama_8b(prompt, param)



print("## payload: ") 
pretty_print_json(pay_load)




## payload: 
{
    "inputs": "\uc0dd\uc131\ud615 AI \ub294",
    "parameters": {
        "do_sample": true,
        "max_new_tokens": 256
    }
}


In [66]:
import time
s = time.perf_counter()

response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    

elapsed_async = time.perf_counter() - s


response = llama3_output_parser(response)




from termcolor import colored

print("## inference esponse: ")                      
print_ww(colored(response, "green"))                         





## inference esponse: 
생성형 AI 는 사람의 업무를 자동화하기 위해 개발된 AI로, 일반적으로 씨피유(CPU)와 자원 활용이 다양하게 가능합니다. 대상하는 업무의 종류가 지도기반 이미지 분류,
단어 처리, 음성 등이 있습니다. 이러한 유형으로는 오크타드 AI 기술을 적용하는데, 보다 직관적인 인간 감정 표현과 컴퓨터가 인간과 같은 진보하는 유머 능력에 대한 자극을 제공할
수 있습니다. Apple가 개발한 오크타드는 당대 해독되지 않은 언어를 복사하고 스스로 언어를 배울 수 있는 메커니즘을 갖추로써 다른 언어들의 학습에 대한 저평 수준을 출발한
대표적인 예시입니다.

갈매기 스퀘이 1:1 대화 형식으로 대화한 것과 같으며, 학습에 도움이 될 수 있는 특정 전략들은 형성과 사람의 지능 사이에 높은 연관성이 있습니다. 모델들은 모든 대화를 완료하는
것과 같은 증가된 압력을 받으며 덧붙여 질문하는 것에도 더 많을 것입니다. 또한, AI들은 특정 주제


## 추론 요약 

In [74]:
prompt = "다음 글을 30자 아내로 요약해줘.\n, 대학교육협의회는 이르면 이번 주 내년도 대학입학 전형시행계획을 심의하고 최종 의대 모집 규모를 승인합니다.\n\
교육부 관계자는 수험생과 학부모를 위해 가능한 빨리 모집 규모를 확정할 필요가 있다며 구체적인 심사 일정을 논의하겠다고 설명했습니다.\
최종 승인 결과에 따라 내년도 의대 정원은 올해보다 천500명 안팎으로 늘어납니다.\
대통령실은 의대 증원이 일단락됐다며 각 대학이 의대 증원을 담은 학칙 개정을 조속히 완료해달라고 당부했습니다.\
동시에 의료계를 향해선 의대 증원 유예와 백지화라는 실현 불가능한 전제조건을 접고 대화하자고 제안했습니다.\
그러나 의료계 반발은 여전합니다.\
당장 의대생 단체는 정부가 귀를 닫고 복귀만을 호소하는 오만한 태도를 보이고 있다고 비판했습니다.\
전공의들 역시 복귀하지 않겠다는 뜻을 분명히 하고 있습니다.\
대한의사협회도 오는 22일 비공개 긴급회의를 열고 의대 교수 등과 구체적인 대응 수단을 논의합니다.\
의료계가 기댔던 의대 증원 집행정지 신청이 기각·각하된 만큼 다음 단계 대응 로드맵을 마련하겠다는 겁니다.\n\n"

param = {"do_sample": True, "max_new_tokens": 512}
pay_load = create_payload_llama_8b(prompt, param)



print("## payload: ") 
pretty_print_json(pay_load)




## payload: 
{
    "inputs": "\ub2e4\uc74c \uae00\uc744 30\uc790 \uc544\ub0b4\ub85c \uc694\uc57d\ud574\uc918.\n, \ub300\ud559\uad50\uc721\ud611\uc758\ud68c\ub294 \uc774\ub974\uba74 \uc774\ubc88 \uc8fc \ub0b4\ub144\ub3c4 \ub300\ud559\uc785\ud559 \uc804\ud615\uc2dc\ud589\uacc4\ud68d\uc744 \uc2ec\uc758\ud558\uace0 \ucd5c\uc885 \uc758\ub300 \ubaa8\uc9d1 \uaddc\ubaa8\ub97c \uc2b9\uc778\ud569\ub2c8\ub2e4.\n\uad50\uc721\ubd80 \uad00\uacc4\uc790\ub294 \uc218\ud5d8\uc0dd\uacfc \ud559\ubd80\ubaa8\ub97c \uc704\ud574 \uac00\ub2a5\ud55c \ube68\ub9ac \ubaa8\uc9d1 \uaddc\ubaa8\ub97c \ud655\uc815\ud560 \ud544\uc694\uac00 \uc788\ub2e4\uba70 \uad6c\uccb4\uc801\uc778 \uc2ec\uc0ac \uc77c\uc815\uc744 \ub17c\uc758\ud558\uaca0\ub2e4\uace0 \uc124\uba85\ud588\uc2b5\ub2c8\ub2e4.\ucd5c\uc885 \uc2b9\uc778 \uacb0\uacfc\uc5d0 \ub530\ub77c \ub0b4\ub144\ub3c4 \uc758\ub300 \uc815\uc6d0\uc740 \uc62c\ud574\ubcf4\ub2e4 \ucc9c500\uba85 \uc548\ud30e\uc73c\ub85c \ub298\uc5b4\ub0a9\ub2c8\ub2e4.\ub300\ud1b5\ub839\uc2e4\uc740 

In [75]:
import time
s = time.perf_counter()

response = invoke_endpoint_sagemaker(endpoint_name = endpoint_name, 
                         pay_load = pay_load)    

elapsed_async = time.perf_counter() - s


response = llama3_output_parser(response)




from termcolor import colored

print("## inference esponse: ")                      
print_ww(colored(response, "green"))                         





## inference esponse: 
다음 글을 30자 아내로 요약해줘.
, 대학교육협의회는 이르면 이번 주 내년도 대학입학 전형시행계획을 심의하고 최종 의대 모집 규모를 승인합니다.
교육부 관계자는 수험생과 학부모를 위해 가능한 빨리 모집 규모를 확정할 필요가 있다며 구체적인 심사 일정을 논의하겠다고 설명했습니다.최종 승인 결과에 따라 내년도 의대 정원은
올해보다 천500명 안팎으로 늘어납니다.대통령실은 의대 증원이 일단락됐다며 각 대학이 의대 증원을 담은 학칙 개정을 조속히 완료해달라고 당부했습니다.동시에 의료계를 향해선 의대
증원 유예와 백지화라는 실현 불가능한 전제조건을 접고 대화하자고 제안했습니다.그러나 의료계 반발은 여전합니다.당장 의대생 단체는 정부가 귀를 닫고 복귀만을 호소하는 오만한 태도를
보이고 있다고 비판했습니다.전공의들 역시 복귀하지 않겠다는 뜻을 분명히 하고 있습니다.대한의사협회도 오는 22일 비공개 긴급회의를 열고 의대 교수 등과 구체적인 대응 수단을
논의합니다.의료계가 기댔던 의대 증원 집행정지 신청이 기각·각하된 만큼 다음 단계 대응 로드맵을 마련하겠다는 겁니다.

국내 의과대학의 정원 확대를 둘러싸고 꾸준히 발생하던 의대안 도입 및 의사국제등록 과정과 관련하여 이번 논쟁이 뜨거워졌습니다.한국의 최초 유료의과대학 개념에는 많은 이들이 동의하고
있지만, 국내 의과대학 세부 프로그램 취지와 과정에 대해서는 여전히 불타는 의견이 분분합니다.의대생들은 의사국제등록과 정부 정책 간극을 메워줄 수 있는 제도적 대응책을 모색하기도
하고, 해외 의사 연수에 대한 무관배제 검토를 위해 머리를 맞댑니다.내년 2월부터 시행될 해외 의사국제등록에 관한 법 및 의사국제등록 제도개선을 위한 법안 개정이 약속될 때까지
국외 의사유치와 관련해서는 또 다른 추가적인 우려가 생길 것으로 예상되고 있습니다. 이에 세계적으로 인정받는 우리나라 의료의 질 향상과 발전을 위해서는 외점 의사 풀을 염두에 둬
제대로 된 대응 방안을 마련해야 한다는 목소리도 높아서 대화,

# 5. 엔드포인트 삭제

In [ ]:
def delete_endpoint_model(endpoint_name,llm_model ):
    sess.delete_endpoint(endpoint_name)
    sess.delete_endpoint_config(endpoint_name)
    llm_model.delete_model()

delete_endpoint_model(endpoint_name,huggingface_model)

